In [1]:
import numpy as np
import pandas as pd
import os
import re
from tqdm import tqdm
import requests
from io import StringIO
from os import listdir
from os.path import isfile, join
import datetime as dt
from datetime import datetime, timedelta

import gridstatus

In [2]:
def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f



In [3]:
FILE_PATH = "/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data"

In [4]:
ISOS = ["ISONE"]

In [37]:
ISO_files = {}
ISO_data = {}
ISO_final = {}

#load in
for ISO in ISOS:
    ISO_files[ISO] = listdir_nohidden(f"{FILE_PATH}/{ISO}/old")
    ISO_data[ISO] = {}
    for file in tqdm(ISO_files[ISO]):
        compiled = re.compile(re.escape(f"{ISO}_"), re.IGNORECASE)

        ISO_data[ISO][compiled.sub("", file).replace(".csv","")] = pd.read_csv(f"{FILE_PATH}/{ISO}/old/{file}")

19it [00:01, 10.94it/s]


In [38]:
#add merge datetime
titles = {}
for key in ISO_data:
    print(key)
    titles[key]= []
    for k in ISO_data[key]:
        try:
            ISO_data[key][k]["time_tomerge"] = pd.to_datetime(ISO_data[key][k].iloc[:,0], utc=True)
        except:
            ISO_data[key][k] = ISO_data[key][k][ISO_data[key][k].index % 24 != 0]
            ISO_data[key][k]["time_tomerge"] = pd.to_datetime(ISO_data[key][k].iloc[:,0], utc=True)
            print(type(ISO_data[key][k]["time_tomerge"]))

ISONE


In [39]:
ISO_final = {}
def clean_final(ISO: str, df: str):
    ISO_data[ISO][df].columns = ISO_data[ISO][df].columns.str.lower()
    no_date = [col for col in ISO_data[ISO][df].columns if "date" not in col]
    ISO_data[ISO][df] = ISO_data[ISO][df][no_date]
    try:
        ISO_data[ISO][df] = ISO_data[ISO][df].drop("unnamed: 0")
        ISO_data[ISO][df] = ISO_data[ISO][df].drop("hour")
    except:
        pass
    ISO_data[ISO][df] = ISO_data[ISO][df].sort_values(by="time_tomerge")
    ISO_data[ISO][df] = ISO_data[ISO][df].set_index("time_tomerge")
    ISO_final[ISO] = ISO_data[ISO][df]
    for column in ISO_final[ISO].columns:
        if str(ISO).lower() in column:
            ISO_final[ISO] = ISO_final[ISO].rename(columns={column: column.replace(f"{str(ISO).lower()}_","")})
    print(ISO_final[ISO])

In [40]:
for iso in ISOS:
    clean_final(iso, "all_data")

                           congestion_da  congestion_rt  energy_da  energy_rt  \
time_tomerge                                                                    
2014-01-01 05:00:00+00:00            0.0        0.00000      62.04     117.43   
2014-01-01 06:00:00+00:00            0.0        0.00000      54.65     148.38   
2014-01-01 07:00:00+00:00            0.0        0.00000      38.40      97.06   
2014-01-01 08:00:00+00:00            0.0        0.00000      38.02      38.54   
2014-01-01 09:00:00+00:00            0.0        0.00000      37.48      79.01   
...                                  ...            ...        ...        ...   
2019-01-01 00:00:00+00:00            0.0       -0.01875      69.19      24.96   
2019-01-01 01:00:00+00:00            0.0       -0.01125      68.83      21.33   
2019-01-01 02:00:00+00:00            0.0        0.00000      59.21      22.37   
2019-01-01 03:00:00+00:00            0.0        0.00000      53.27      24.91   
2019-01-01 04:00:00+00:00   

In [41]:
ISO_final["ISONE"].columns

Index(['congestion_da', 'congestion_rt', 'energy_da', 'energy_rt',
       'exchange_rt', 'unnamed: 0', 'gas_price', 'gen_coal', 'gen_hydro',
       'gen_landfillgas', 'gen_gas', 'gen_nuclear', 'gen_oil', 'gen_other',
       'gen_refuse', 'gen_solar', 'gen_wind', 'gen_wood', 'load_rt',
       'losses_da', 'losses_rt', 'regulation_price', 'regulation_quantity',
       'reserves_price', 'reserves_quantity', 'gen_solar2', 'gen_wind2'],
      dtype='object')

In [42]:
from datetime import datetime, timedelta
def date_range(start_date, end_date):
    date_list = []
    current = start_date
    end_dt = end_date
    for n in range(int((end_dt - current).days)+1):
        d = current + timedelta(n)
        date_list.append(d.strftime('%Y-%m-%d'))
    return [date.replace("-", "") for date in date_list]




##FOR COMBINING DATA
ISO_done = ["ISONE"]
FILE_PATH = "/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data/ISONE/new"
for file in (listdir_nohidden(FILE_PATH)):
    if file in ["wind_gen", "solar_gen"]:
        dfs = []
        try:
            for f in tqdm(listdir_nohidden(f"{FILE_PATH}/{file}")):
                dfs.append(pd.read_csv(f"{FILE_PATH}/{file}/{f}", skiprows=4).iloc[1:,:])
        except:
            for f in tqdm(listdir_nohidden(f"{FILE_PATH}/{file}")):
                print(f)
                dfs.append(pd.read_csv(f"{FILE_PATH}/{file}/{f}"))
        temp = pd.concat(dfs)
        os.makedirs(f"{FILE_PATH}/combined", exist_ok=True)
        temp.to_csv(f"{FILE_PATH}/combined/{file}.csv")

In [11]:
energy_da = pd.read_csv("/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data/ISONE/new/combined/energy_da.csv")
energy_da = energy_da[energy_da["Location Name"] == ".H.INTERNAL_HUB"]
energy_rt = pd.read_csv("/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data/ISONE/new/combined/energy_rt.csv")
energy_rt = energy_rt[energy_rt["Location Name"] == ".H.INTERNAL_HUB"]


/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3442: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [12]:
def clean_frame(energy: pd.DataFrame, group_cols: list, to_keep_cols: list, date_format: str):

    #remove the daylight savings hours
    energy= energy[energy[group_cols[1]] != "02X"]

    #groupby and average
    energy_mean = pd.DataFrame(energy.groupby(group_cols)[to_keep_cols].mean())

    #construct the times
    time_tomerge = []

    for i in tqdm(range(len(energy_mean))):
        try:
            time_tomerge.append(datetime.strptime(str(energy_mean.index[i][0]), date_format) + timedelta(hours = int(energy_mean.index[i][1])))
        except:
            print(energy_mean.index[i])


    energy_mean['time_tomerge'] = time_tomerge
    energy_mean = energy_mean.sort_values(by='time_tomerge')
    energy_mean = energy_mean.reset_index(drop=True)

    return energy_mean

In [13]:
energy_da = clean_frame(energy_da, ['Date', "Hour Ending"], ['Locational Marginal Price', 'Energy Component', 'Congestion Component', 'Marginal Loss Component'],'%m/%d/%Y')
energy_rt = clean_frame(energy_rt, ['Date', "Hour Ending"], ['Locational Marginal Price', 'Energy Component', 'Congestion Component', 'Marginal Loss Component'], '%m/%d/%Y')


100%|██████████| 35012/35012 [00:01<00:00, 22364.72it/s]


In [14]:
print(energy_rt)

       Locational Marginal Price  Energy Component  Congestion Component  \
0                          35.65             35.42                  0.04   
1                          38.63             38.38                  0.00   
2                          21.23             21.09                  0.00   
3                          18.45             18.29                  0.03   
4                           7.54              7.48                  0.01   
...                          ...               ...                   ...   
35007                      26.36             26.27                  0.00   
35008                      25.38             25.31                  0.00   
35009                      31.80             31.69                  0.00   
35010                      44.29             44.16                  0.00   
35011                      58.75             58.63                  0.00   

       Marginal Loss Component        time_tomerge  
0                         0.19 201

In [43]:
exchange = pd.read_csv("/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data/ISONE/new/combined/exchange.csv")
exchange = clean_frame(exchange, ["Local Date", "Local Hour Ending"], ["Actual Interchange", "Purchases", "Sales"], '%Y-%m-%d')
print(exchange)

100%|██████████| 35034/35034 [00:01<00:00, 17776.39it/s]


       Actual Interchange   Purchases       Sales        time_tomerge
0             -458.916667 -619.458333  160.541667 2019-01-01 01:00:00
1             -454.416667 -603.875000  149.458333 2019-01-01 02:00:00
2             -482.083333 -613.625000  131.541667 2019-01-01 03:00:00
3             -486.875000 -679.625000  192.750000 2019-01-01 04:00:00
4             -516.666667 -683.166667  166.500000 2019-01-01 05:00:00
...                   ...         ...         ...                 ...
35029         -474.375000 -683.166667  208.791667 2022-12-31 20:00:00
35030         -439.666667 -669.833333  230.166667 2022-12-31 21:00:00
35031         -330.916667 -512.583333  181.666667 2022-12-31 22:00:00
35032         -309.666667 -503.958333  194.291667 2022-12-31 23:00:00
35033         -314.375000 -508.208333  193.833333 2023-01-01 00:00:00

[35034 rows x 4 columns]


In [44]:
generation = pd.read_csv("/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data/ISONE/new/combined/generation.csv")

type(generation['Time'][0])

str

In [45]:
generation = pd.read_csv("/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data/ISONE/new/combined/generation.csv")
generation = generation.drop(columns = ["Unnamed: 0", "Unnamed: 0.1"])
generation['time_tomerge'] = pd.to_datetime(generation['Time'], utc=True).dt.floor("H")
generation['time_tomerge'] = generation['time_tomerge'] - timedelta(hours=5)
generation = pd.DataFrame(generation.groupby(["time_tomerge"])[generation.columns[1:-1]].mean())
generation = generation.reset_index()
print(generation)

                   time_tomerge        Hydro  Landfill Gas  Natural Gas  \
0     2019-01-01 00:00:00+00:00  1040.285714     25.285714  2389.571429   
1     2019-01-01 01:00:00+00:00   880.333333     25.666667  2421.166667   
2     2019-01-01 02:00:00+00:00   902.750000     26.000000  2061.250000   
3     2019-01-01 03:00:00+00:00   942.000000     26.000000  1966.125000   
4     2019-01-01 04:00:00+00:00   954.285714     25.714286  1859.571429   
...                         ...          ...           ...          ...   
31706 2022-12-31 19:00:00+00:00  1183.888889     36.000000  5088.222222   
31707 2022-12-31 20:00:00+00:00  1142.666667     36.000000  4716.222222   
31708 2022-12-31 21:00:00+00:00  1141.750000     36.000000  5001.500000   
31709 2022-12-31 22:00:00+00:00  1173.666667     35.888889  4296.666667   
31710 2022-12-31 23:00:00+00:00  1210.375000     35.875000  3658.375000   

           Nuclear     Other      Refuse  Solar        Wind        Wood  Oil  \
0      4016.714286 

In [46]:
load = pd.read_csv("/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data/ISONE/new/combined/load.csv")
load = load.dropna()
print(load)
load = clean_frame(load, ['Date', "Hour Ending"], ['Total Load'],'%m/%d/%Y')
print(load)

       Unnamed: 0  H        Date Hour Ending  Total Load
0               1  D  07/16/2019          01    13319.45
1               2  D  07/16/2019          02    13012.74
2               3  D  07/16/2019          03    12572.27
3               4  D  07/16/2019          04    12390.46
4               5  D  07/16/2019          05    12192.45
...           ... ..         ...         ...         ...
35188         308  D  02/28/2019          20    17110.14
35189         309  D  02/28/2019          21    16597.57
35190         310  D  02/28/2019          22    15759.28
35191         311  D  02/28/2019          23    14633.02
35192         312  D  02/28/2019          24    13828.94

[35066 rows x 5 columns]


100%|██████████| 35060/35060 [00:01<00:00, 25226.16it/s]

       Total Load        time_tomerge
0        11613.92 2019-01-01 01:00:00
1        11496.66 2019-01-01 02:00:00
2        11368.76 2019-01-01 03:00:00
3        11270.26 2019-01-01 04:00:00
4        11309.61 2019-01-01 05:00:00
...           ...                 ...
35055    13362.98 2022-12-31 20:00:00
35056    12769.96 2022-12-31 21:00:00
35057    12376.44 2022-12-31 22:00:00
35058    11509.74 2022-12-31 23:00:00
35059    10922.34 2023-01-01 00:00:00

[35060 rows x 2 columns]


In [47]:
reserve_prices = pd.read_csv("/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data/ISONE/new/combined/reserve_price_quantity.csv")
reserve_prices = reserve_prices.dropna()
reserve_prices = clean_frame(reserve_prices, ['Unnamed: 1', 'Unnamed: 2'], ["Ten-Minute Spinning Reserve Clearing Price", "Ten-Minute Reserve Clearing Price", "Total Reserve Clearing Price", "TMSR Designated MW","TMNSR Designated MW", "TMOR Designated MW"],'%Y-%m-%d')
print(reserve_prices)

100%|██████████| 35052/35052 [00:01<00:00, 23991.92it/s]


       Ten-Minute Spinning Reserve Clearing Price  \
0                                           17.10   
1                                            0.00   
2                                            0.00   
3                                            0.00   
4                                            0.00   
...                                           ...   
35047                                        0.00   
35048                                        0.00   
35049                                        0.00   
35050                                        2.54   
35051                                        1.56   

       Ten-Minute Reserve Clearing Price  Total Reserve Clearing Price  \
0                                    0.0                           0.0   
1                                    0.0                           0.0   
2                                    0.0                           0.0   
3                                    0.0                           

In [48]:
solar_gen = pd.read_csv("/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data/ISONE/new/combined/solar_gen.csv")
wind_gen = pd.read_csv("/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data/ISONE/new/combined/wind_gen.csv")

solar_gen = clean_frame(solar_gen, ["local_day", "LOCAL_HOUR_END"], ["tot_solar_mwh", "_FREQ_"], "%d-%b-%y")
wind_gen = clean_frame(wind_gen, ["local_day", "local_hour_end"], ["tot_wind_mwh", "_FREQ_"], "%d-%b-%y")

100%|██████████| 35060/35060 [00:01<00:00, 21628.88it/s]


In [49]:
energy_rt = energy_rt.rename(columns={"Energy Component": "energy_rt", "Congestion Component": "congestion_rt", "Marginal Loss Component": "losses_rt"})
energy_rt = energy_rt.drop(columns = ["Locational Marginal Price"])

energy_rt

KeyError: "['Locational Marginal Price'] not found in axis"

In [22]:
energy_da = energy_da.rename(columns={"Energy Component": "energy_da", "Congestion Component": "congestion_da", "Marginal Loss Component": "losses_da"})
energy_da = energy_da.drop(columns = ["Locational Marginal Price"])
energy_da

,energy_da,congestion_da,losses_da,time_tomerge
0,25.57,0.02,0.04,2019-01-01 01:00:00
1,20.44,0.01,0.04,2019-01-01 02:00:00
2,18.84,0.01,0.01,2019-01-01 03:00:00
3,19.09,0.01,0.02,2019-01-01 04:00:00
4,17.78,0.00,0.02,2019-01-01 05:00:00
...,...,...,...,...
35055,38.97,0.00,0.04,2022-12-31 20:00:00
35056,38.88,0.00,0.00,2022-12-31 21:00:00
35057,38.73,0.00,-0.05,2022-12-31 22:00:00
35058,38.69,0.00,-0.04,2022-12-31 23:00:00


In [50]:
generation = generation.rename(columns = {"Solar": "gen_solar", "Wind": "gen_wind", "Hydro": "gen_hydro", "Coal": "gen_coal", "Nuclear": "gen_nuclear", "Natural Gas": "gen_gas", "Landfill Gas": "gen_landfillgas", "Refuse": "gen_refuse", "Wood": "gen_wood", "Oil": "gen_oil", "Other": "gen_other"})
generation

,time_tomerge,gen_hydro,gen_landfillgas,gen_gas,gen_nuclear,gen_other,gen_refuse,gen_solar,gen_wind,gen_wood,gen_oil,gen_coal
0,2019-01-01 00:00:00+00:00,1040.285714,25.285714,2389.571429,4016.714286,NaN,358.142857,0.0,671.142857,368.857143,NaN,0.000000
1,2019-01-01 01:00:00+00:00,880.333333,25.666667,2421.166667,4017.333333,NaN,361.666667,0.0,751.666667,374.500000,NaN,0.000000
2,2019-01-01 02:00:00+00:00,902.750000,26.000000,2061.250000,4016.500000,NaN,357.125000,0.0,783.875000,369.000000,NaN,0.000000
3,2019-01-01 03:00:00+00:00,942.000000,26.000000,1966.125000,4016.125000,NaN,361.625000,0.0,757.625000,365.375000,NaN,0.000000
4,2019-01-01 04:00:00+00:00,954.285714,25.714286,1859.571429,4016.142857,NaN,355.714286,0.0,648.142857,370.857143,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
31706,2022-12-31 19:00:00+00:00,1183.888889,36.000000,5088.222222,3371.444444,6.777778,323.444444,1.0,318.555556,231.444444,NaN,0.222222
31707,2022-12-31 20:00:00+00:00,1142.666667,36.000000,4716.222222,3371.444444,5.444444,345.666667,1.0,271.666667,231.777778,NaN,0.000000
31708,2022-12-31 21:00:00+00:00,1141.750000,36.000000,5001.500000,3370.750000,4.750000,342.250000,1.0,238.125000,231.125000,NaN,0.000000
31709,2022-12-31 22:00:00+00:00,1173.666667,35.888889,4296.666667,3371.444444,6.888889,333.555556,1.0,207.666667,232.222222,NaN,0.000000


In [51]:
load = load.rename(columns = {"Total Load": "load_rt"})
load

,load_rt,time_tomerge
0,11613.92,2019-01-01 01:00:00
1,11496.66,2019-01-01 02:00:00
2,11368.76,2019-01-01 03:00:00
3,11270.26,2019-01-01 04:00:00
4,11309.61,2019-01-01 05:00:00
...,...,...
35055,13362.98,2022-12-31 20:00:00
35056,12769.96,2022-12-31 21:00:00
35057,12376.44,2022-12-31 22:00:00
35058,11509.74,2022-12-31 23:00:00


In [52]:
reserve_prices["reserves_price"] = reserve_prices[['Ten-Minute Spinning Reserve Clearing Price',
       'Ten-Minute Reserve Clearing Price', 'Total Reserve Clearing Price']].mean(axis=1)
reserve_prices["reserves_quantity"] = reserve_prices[['TMSR Designated MW', 'TMNSR Designated MW', 'TMOR Designated MW']].mean(axis=1)
reserve_prices = reserve_prices.drop(columns = ['Ten-Minute Spinning Reserve Clearing Price',
       'Ten-Minute Reserve Clearing Price', 'Total Reserve Clearing Price', 'TMSR Designated MW', 'TMNSR Designated MW', 'TMOR Designated MW'])
reserve_prices

,time_tomerge,reserves_price,reserves_quantity
0,2019-01-01 01:00:00,5.700000,377.893000
1,2019-01-01 02:00:00,0.000000,414.222917
2,2019-01-01 03:00:00,0.000000,489.601417
3,2019-01-01 04:00:00,0.000000,516.881250
4,2019-01-01 05:00:00,0.000000,531.750750
...,...,...,...
35047,2022-12-31 20:00:00,0.000000,431.456333
35048,2022-12-31 21:00:00,0.000000,427.646500
35049,2022-12-31 22:00:00,0.000000,418.256917
35050,2022-12-31 23:00:00,0.846667,394.830667


In [53]:
wind_gen['gen_wind2'] = wind_gen['tot_wind_mwh']
wind_gen = wind_gen.drop(columns = ['tot_wind_mwh', '_FREQ_'])
wind_gen

,time_tomerge,gen_wind2
0,2019-01-01 01:00:00,683.931
1,2019-01-01 02:00:00,764.107
2,2019-01-01 03:00:00,799.717
3,2019-01-01 04:00:00,770.408
4,2019-01-01 05:00:00,665.267
...,...,...
35055,2022-12-31 20:00:00,322.118
35056,2022-12-31 21:00:00,283.284
35057,2022-12-31 22:00:00,240.049
35058,2022-12-31 23:00:00,216.036


In [54]:
solar_gen['gen_solar2'] = solar_gen['tot_solar_mwh']
solar_gen = solar_gen.drop(columns = ['tot_solar_mwh', '_FREQ_'])
solar_gen.head(10)

,time_tomerge,gen_solar2
0,2019-01-01 01:00:00,NaN
1,2019-01-01 02:00:00,NaN
2,2019-01-01 03:00:00,NaN
3,2019-01-01 04:00:00,NaN
4,2019-01-01 05:00:00,NaN
5,2019-01-01 06:00:00,NaN
6,2019-01-01 07:00:00,2.196
7,2019-01-01 08:00:00,7.978
8,2019-01-01 09:00:00,56.889
9,2019-01-01 10:00:00,140.145


In [55]:
generation["time_tomerge"] = pd.to_datetime(generation["time_tomerge"])

In [57]:
exchange = exchange.rename(columns= {"Actual Interchange": "exchange_rt", "Purchases": "exchange_negative", "Sales":"exchange_positive"})

In [58]:
dfs = [energy_rt, energy_da, exchange, generation, load, wind_gen, solar_gen]

In [59]:
total = dfs[0]
print(total.columns)
total["time_tomerge"] = pd.to_datetime(total["time_tomerge"],utc =True)
for df in tqdm(dfs[1:]):
    df["time_tomerge"] = pd.to_datetime(df["time_tomerge"],utc =True)
    total = pd.merge(total, df, on="time_tomerge", how = "outer")
total = total.sort_values(by="time_tomerge")
total = total.drop_duplicates('time_tomerge')
print(total)

Index(['energy_rt', 'congestion_rt', 'losses_rt', 'time_tomerge'], dtype='object')


100%|██████████| 6/6 [00:00<00:00, 24.73it/s]

       energy_rt  congestion_rt  losses_rt              time_tomerge  \
35060        NaN            NaN        NaN 2019-01-01 00:00:00+00:00   
0          35.42           0.04       0.19 2019-01-01 01:00:00+00:00   
1          38.38           0.00       0.25 2019-01-01 02:00:00+00:00   
2          21.09           0.00       0.14 2019-01-01 03:00:00+00:00   
3          18.29           0.03       0.13 2019-01-01 04:00:00+00:00   
...          ...            ...        ...                       ...   
35007      26.27           0.00       0.09 2022-12-31 20:00:00+00:00   
35008      25.31           0.00       0.07 2022-12-31 21:00:00+00:00   
35009      31.69           0.00       0.11 2022-12-31 22:00:00+00:00   
35010      44.16           0.00       0.13 2022-12-31 23:00:00+00:00   
35011      58.63           0.00       0.12 2023-01-01 00:00:00+00:00   

       energy_da  congestion_da  losses_da  exchange_rt  exchange_negative  \
35060        NaN            NaN        NaN          NaN  

In [60]:
ISO_final["ISONE"] = ISO_final["ISONE"].reset_index()
ISO_final["ISONE"]["time_tomerge"] = ISO_final["ISONE"]["time_tomerge"] - timedelta(hours=5)

In [61]:
final = pd.concat([ISO_final["ISONE"], total])
final.drop(columns=["gen_solar2", "gen_wind2"])

,time_tomerge,congestion_da,congestion_rt,energy_da,energy_rt,exchange_rt,unnamed: 0,gas_price,gen_coal,gen_hydro,...,gen_wood,load_rt,losses_da,losses_rt,regulation_price,regulation_quantity,reserves_price,reserves_quantity,exchange_negative,exchange_positive
0,2014-01-01 00:00:00+00:00,0.0,0.0,62.04,117.43,-3134.000000,2014-01-01 00:00:00-05:00,20.151667,NaN,NaN,...,NaN,14312.05,-0.17375,-0.05125,38.92,38.92,0.0,526.100,NaN,NaN
1,2014-01-01 01:00:00+00:00,0.0,0.0,54.65,148.38,-3168.000000,2014-01-01 01:00:00-05:00,20.151667,NaN,NaN,...,NaN,14235.98,-0.15500,-0.02375,59.24,59.24,0.0,441.233,NaN,NaN
2,2014-01-01 02:00:00+00:00,0.0,0.0,38.40,97.06,-3550.000000,2014-01-01 02:00:00-05:00,20.151667,NaN,NaN,...,NaN,14364.54,-0.07500,-0.02875,43.18,43.18,0.0,541.325,NaN,NaN
3,2014-01-01 03:00:00+00:00,0.0,0.0,38.02,38.54,-3883.000000,2014-01-01 03:00:00-05:00,20.151667,NaN,NaN,...,NaN,14258.79,-0.09625,-0.03625,81.93,81.93,0.0,641.900,NaN,NaN
4,2014-01-01 04:00:00+00:00,0.0,0.0,37.48,79.01,-3806.000000,2014-01-01 04:00:00-05:00,20.151667,NaN,NaN,...,NaN,14300.04,-0.08875,-0.08000,49.27,49.27,0.0,651.575,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35007,2022-12-31 20:00:00+00:00,0.0,0.0,38.97,26.27,-474.375000,NaN,NaN,0.0,1142.666667,...,231.777778,13362.98,0.04000,0.09000,NaN,NaN,NaN,NaN,-683.166667,208.791667
35008,2022-12-31 21:00:00+00:00,0.0,0.0,38.88,25.31,-439.666667,NaN,NaN,0.0,1141.750000,...,231.125000,12769.96,0.00000,0.07000,NaN,NaN,NaN,NaN,-669.833333,230.166667
35009,2022-12-31 22:00:00+00:00,0.0,0.0,38.73,31.69,-330.916667,NaN,NaN,0.0,1173.666667,...,232.222222,12376.44,-0.05000,0.11000,NaN,NaN,NaN,NaN,-512.583333,181.666667
35010,2022-12-31 23:00:00+00:00,0.0,0.0,38.69,44.16,-309.666667,NaN,NaN,0.0,1210.375000,...,232.625000,11509.74,-0.04000,0.13000,NaN,NaN,NaN,NaN,-503.958333,194.291667


In [62]:
final.to_csv("/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/ISOs/ISONE2.csv")
